# MLflow Tracing para LLMs con LangChain y Google Gemini

En este notebook exploraremos las nuevas capacidades de **MLflow Tracing** para el seguimiento y observabilidad de modelos de lenguaje (LLMs). 

## ¿Qué es MLflow Tracing?

MLflow Tracing es un sistema de observabilidad que captura automáticamente:
- **Prompts y respuestas** enviados/recibidos del modelo
- **Latencias** de cada llamada
- **Tokens utilizados** (input, output, total)
- **Metadatos** (temperatura, modelo, etc.)
- **Excepciones** si ocurren errores

## Integraciones Soportadas

MLflow soporta **40+ integraciones** incluyendo:
- **Frameworks de Agentes:** LangChain, LangGraph, DSPy, CrewAI, LlamaIndex
- **Proveedores de Modelos:** OpenAI, Anthropic, Google Gemini, Mistral, Ollama

## ¿Qué aprenderemos?

1. Usar `mlflow.gemini.autolog()` para tracing directo con Google GenAI SDK
2. Usar `mlflow.langchain.autolog()` para tracing con LangChain
3. Visualizar trazas en la UI de MLflow
4. Comparar diferentes configuraciones de prompts

## 1. Instalación de Dependencias

Instalamos las librerías necesarias. Nota: Se requiere MLflow >= 2.14.0 para tracing con LangChain.

In [1]:
# Instalamos las dependencias necesarias
!pip install "mlflow>=2.14.0" langchain langchain-google-genai google-generativeai python-dotenv

  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
   ---------------------------------------- 0.0/719.4 kB ? eta -:--:--
   ---------------------------------------- 719.4/719.4 kB 4.2 MB/s  0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ----------------------- ---------------- 0.8/1.3 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 4.5 MB/s  0:00:00
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
   ---------------------------------------- 0.0/14.9 MB ? eta 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 2. Configuración Inicial

In [3]:
import os 
from dotenv import load_dotenv
import mlflow 

load_dotenv()

MLFLOW_TRACKING_URI = 'http://localhost:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY')

---
# PARTE 1: MLflow Tracing con Google GenAI SDK Directo

Primero veremos cómo usar `mlflow.gemini.autolog()` para tracing automático con el SDK nativo de Google.

In [4]:
import google.generativeai as genai 

genai.configure(api_key=GOOGLE_API_KEY)

c:\Users\julen\anaconda3\envs\ia4-mlops\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\julen\AppData\Local\Temp\ipykernel_11064\764567260.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [ ]:
model = genai.GenerativeModel('gemini-2.5-flash-lite')

In [7]:
response = model.generate_content('Puedes explicarme las aplicaciones que tiene mlflow')

In [8]:
response.text

'¡Claro! MLflow es una plataforma de código abierto que te ayuda a gestionar todo el ciclo de vida del machine learning. Sus aplicaciones son muy amplias y se centran en resolver los desafíos comunes que enfrentan los científicos de datos e ingenieros de ML.\n\nAquí te explico las principales aplicaciones de MLflow, divididas por sus componentes principales:\n\n**1. MLflow Tracking:**\n\nEsta es la aplicación más fundamental de MLflow y se centra en **registrar, visualizar y comparar experimentos de ML**.\n\n*   **Seguimiento de Experimentos de ML:**\n    *   **Registro de Métricas y Parámetros:** Guarda automáticamente todas las métricas de rendimiento (precisión, F1-score, RMSE, etc.) y los hiperparámetros utilizados en cada ejecución de un modelo.\n    *   **Registro de Artefactos:** Guarda los modelos entrenados, archivos de preprocesamiento, visualizaciones, y cualquier otro archivo relevante generado durante el experimento.\n    *   **Visualización de Resultados:** La interfaz de

In [9]:
mlflow.gemini.autolog()

2026/01/26 21:40:15 WARNING mlflow.utils.autologging_utils: MLflow gemini autologging is known to be compatible with 0.8.0 <= google-generativeai <= 0.8.3, but the installed version is 0.8.6. If you encounter errors during autologging, try upgrading / downgrading google-generativeai to a compatible version, or try upgrading MLflow.


In [13]:
chat = model.start_chat(history=[])

response = chat.send_message('Explicame cuando salio el modelo text-davinci-003')

In [14]:
response.text

'El modelo `text-davinci-003` fue lanzado por OpenAI en **noviembre de 2022**.\n\nFue, en su momento, el modelo más avanzado y potente de la serie GPT-3 disponible a través de la API de OpenAI. Se destacó por su capacidad para:\n\n*   Seguir instrucciones complejas de manera más efectiva.\n*   Generar texto más coherente, creativo y de mayor calidad.\n*   Realizar tareas de razonamiento, escritura de código y comprensión de lenguaje natural de manera impresionante.\n\n`text-davinci-003` jugó un papel fundamental en el auge de la IA generativa y fue el modelo de facto para muchas aplicaciones y experimentos antes de que modelos como `gpt-3.5-turbo` y `gpt-4` fueran lanzados y lo superaran en eficiencia, coste y capacidad.'

---
# PARTE 2: MLflow Tracing con LangChain

Ahora veremos cómo usar `mlflow.langchain.autolog()` para tracing automático con LangChain.

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

# Habilitamos el auto-tracing para LangChain
mlflow.langchain.autolog()

# Configuramos el experimento
#mlflow.set_experiment("Langchain-ia4")



In [24]:
# Inicializamos el modelo de LangChain
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
    max_tokens=1024
)

print("Modelo LangChain inicializado")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Modelo LangChain inicializado


### 2.1 Llamada Simple con Mensajes

In [25]:

messages = [
    SystemMessage(content="Eres un experto en LLMOPs con un acento frances"),
    HumanMessage(content="Explicame como puedo gestionar un sistema de kubernetes")
]

response = llm.invoke(messages)

print("Respuesta:")
print(response.content)

Respuesta:
Ah, *mon cher ami*! ¡Qué excelente pregunta! Gestionar un sistema Kubernetes, ¡eso es un arte, una ciencia, una verdadera aventura! Permíteme guiarte a través de este fascinante mundo, con un toque de mi *je ne sais quoi* francés.

Kubernetes, como sabes, es el orquestador de contenedores por excelencia. Su poder reside en su capacidad para automatizar la implementación, el escalado y la gestión de aplicaciones en contenedores. Pero, como cualquier sistema complejo, requiere una gestión cuidadosa y estratégica.

Aquí tienes los pilares fundamentales para gestionar tu sistema Kubernetes de manera efectiva:

### 1. Arquitectura y Diseño: La Base Sólida

Antes de empezar a desplegar, ¡piensa en la arquitectura!

*   **Clúster:** Decide la topología de tu clúster. ¿Será un clúster pequeño para desarrollo, uno mediano para producción, o un gigante para cargas de trabajo masivas? Considera la separación de nodos de control (control plane) y nodos de trabajo (worker nodes) para may

### 2.2 Usando Chains con PromptTemplate

Las chains de LangChain son trazadas automáticamente, capturando cada paso.

In [26]:

template = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en {tema}. Responde de forma {estilo}."),
    ("human", "{pregunta}")
])


chain = template | llm | StrOutputParser()

print("Chain creada: PromptTemplate -> LLM -> StrOutputParser")

Chain creada: PromptTemplate -> LLM -> StrOutputParser


In [27]:

result = chain.invoke({
    "tema": "Kubernetes",
    "estilo": "Divulgativo",
    "pregunta": "Explicame como puedo desplegar modelos de Deep Learning en un cluster the k8s"
})

print("Respuesta de la Chain:")
print(result)

Respuesta de la Chain:
¡Claro que sí! Desplegar modelos de Deep Learning en Kubernetes (k8s) es un tema fascinante y muy potente. Imagina que tienes un modelo de inteligencia artificial súper chulo, entrenado para reconocer gatos en fotos, y quieres que esté disponible para que muchas aplicaciones puedan usarlo al instante, sin importar cuántas peticiones le lleguen. Kubernetes es como el "director de orquesta" que se encarga de que esto funcione de maravilla.

Vamos a desglosarlo de forma sencilla, como si estuviéramos construyendo un pequeño "equipo" para nuestro modelo.

### ¿Por qué usar Kubernetes para modelos de Deep Learning?

Antes de meternos en faena, pensemos por qué es tan buena idea:

*   **Escalabilidad:** Si de repente miles de personas quieren usar tu modelo para reconocer gatos, Kubernetes puede crear más copias de tu modelo automáticamente para atender toda esa demanda. ¡Adiós a los cuellos de botella!
*   **Alta Disponibilidad:** Si una de las copias de tu modelo fal

### 2.3 Comparación de Diferentes Prompts

Ejecutamos la misma pregunta con diferentes system prompts para comparar en MLflow UI.

In [ ]:
# Diferentes versiones de prompts para comparar
prompt_versions = [
    {
        "name": "basico",
        "system": "Eres un asistente útil.",
        "user": "¿Qué es Docker?"
    },
    {
        "name": "experto",
        "system": "Eres un experto en DevOps con 10 años de experiencia. Responde de forma técnica pero accesible.",
        "user": "¿Qué es Docker?"
    },
    {
        "name": "estructurado",
        "system": "Eres un experto en DevOps. Responde usando: 1) Definición, 2) Casos de uso, 3) Ventajas.",
        "user": "¿Qué es Docker?"
    }
]

for version in prompt_versions:
    print(f"\n{'='*50}")
    print(f"Versión: {version['name']}")
    print(f"{'='*50}")
    
    messages = [
        SystemMessage(content=version["system"]),
        HumanMessage(content=version["user"])
    ]
    
response = llm.invoke(messages)
print(response.content[:300] + "...")
    mlflow.log_metric("response_length_chars", len(response.content))
    mlflow.log_metric("response_length_tokens_est", len(response.content.split()))

    # Log response as artifact (best practice)
    response_path = f"response_{version['name']}.txt"
    with open(response_path, "w", encoding="utf-8") as f:
        f.write(response.content)

    mlflow.log_artifact(response_path)

    print(f"\n{'='*50}")
    print(f"Versión: {version['name']}")
    print(f"{'='*50}")
    print(response.content[:300] + "...")


Versión: basico
Docker es una **plataforma de código abierto** que se utiliza para **desarrollar, enviar y ejecutar aplicaciones en contenedores**.

Piensa en un contenedor como una **caja ligera y portátil** que empaqueta todo lo que una aplicación necesita para ejecutarse: el código, las bibliotecas, las herramie...

Versión: experto
¡Excelente pregunta! Docker es una herramienta fundamental en el mundo DevOps y su impacto ha sido enorme. Permíteme explicártelo de forma técnica pero clara.

En esencia, **Docker es una plataforma para desarrollar, enviar y ejecutar aplicaciones en contenedores.**

Vamos a desglosar esto:

### ¿Qu...

Versión: estructurado
¡Excelente pregunta! Como experto en DevOps, te explico qué es Docker de la siguiente manera:

### 1) Definición

**Docker es una plataforma de código abierto que permite automatizar el despliegue, la escalabilidad y la gestión de aplicaciones utilizando tecnologías de virtualización de contenedores...


---
# PARTE 3: Logging Manual Adicional

Además del tracing automático, podemos añadir métricas y artifacts manualmente.

In [ ]:
import time

def experiment_with_run(run_name, system_prompt, user_prompt, temperature=0.7):

    with mlflow.start_run(run_name=run_name):

        mlflow.log_param("model", "")
        mlflow.log_param("temperature", temperature)
        mlflow.log_param("system_prompt", system_prompt[:200])
        mlflow.log_param("user_prompt", user_prompt[:200])
        
        # Creamos modelo con temperatura específica
        llm_temp = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite",
            google_api_key=GOOGLE_API_KEY,
            temperature=temperature
        )
        
        # Medimos latencia
        start = time.time()
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        response = llm_temp.invoke(messages)
        
        latency = time.time() - start
        
        # Loggeamos métricas
        mlflow.log_metric("latency_seconds", latency)
        mlflow.log_metric("response_length", len(response.content))
        
        mlflow.log_text(response.content, "response.txt")
        
        mlflow.log_dict({
            "system_prompt": system_prompt,
            "user_prompt": user_prompt,
            "response": response.content,
            "latency": latency
        }, "interaction.json")
        
        print(f"Run: {run_name}")
        print(f"Latencia: {latency:.2f}s")
        print(f"Respuesta: {response.content[:150]}...")
        
        return response.content

In [ ]:

mlflow.set_experiment("T")

temperatures = [0.0, 0.5, 1.0]
prompt = ""

for temp in temperatures:
    print(f"\n{'='*50}")
    experiment_with_run(
        run_name=f"temp_{temp}",
        system_prompt="",
        user_prompt=prompt,
        temperature=temp
    )

---
# PARTE 4: Tracing Manual con Decorador

Podemos usar `@mlflow.trace` para crear spans personalizados.

In [ ]:
@mlflow.trace
def generate_summary(text: str, max_words: int = 50) -> str:
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"Resume el siguiente texto en máximo {max_words} palabras."),
        ("human", "{text}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"text": text})


@mlflow.trace
def analyze_and_summarize(text: str) -> dict:
    summary = generate_summary(text, max_words=30)
    
    return {
        "original_length": len(text),
        "summary": summary,
        "summary_length": len(summary)
    }

In [ ]:
mlflow.set_experiment("Custom_Tracin")

texto = """
"""

result = analyze_and_summarize(texto)
print(f"Texto original: {result['original_length']} caracteres")
print(f"Resumen: {result['summary']}")

---
# PARTE 5: Evaluación de Respuestas

In [ ]:
import pandas as pd

# Dataset de evaluación
eval_data = pd.DataFrame({
    "question": [
        "¿Qué es un contenedor en Docker?",
        "¿Para qué sirve Kubernetes?",
        "¿Qué es CI/CD?"
    ],
    "expected_keywords": [
        ["software", "dependencias", "aislado"],
        ["orquestación", "contenedores", "escalado"],
        ["integración", "continua", "despliegue"]
    ]
})

print(eval_data)

In [ ]:
mlflow.set_experiment("")

with mlflow.start_run(run_name="evaluation_run"):
    results = []
    
    for idx, row in eval_data.iterrows():
        start = time.time()
        
        messages = [
            SystemMessage(content=""),
            HumanMessage(content=row["question"])
        ]
        response = llm.invoke(messages)
        
        latency = time.time() - start
        
        response_lower = response.content.lower()
        keywords_found = sum(1 for kw in row["expected_keywords"] if kw in response_lower)
        keyword_score = keywords_found / len(row["expected_keywords"])
        
        results.append({
            "question": row["question"],
            "response": response.content,
            "latency": latency,
            "keyword_score": keyword_score
        })
        
        mlflow.log_metric(f"latency_q{idx}", latency)
        mlflow.log_metric(f"keyword_score_q{idx}", keyword_score)
    
    results_df = pd.DataFrame(results)
    mlflow.log_metric("avg_latency", results_df["latency"].mean())
    mlflow.log_metric("avg_keyword_score", results_df["keyword_score"].mean())
    
    mlflow.log_table(results_df, "evaluation_results.json")
    
    print("Resultados de evaluación:")
    print(results_df[["question", "latency", "keyword_score"]])